In [2]:
import numpy as np
import pandas as pd
import re
import os

In [3]:
df = pd.read_csv("combined_file.csv")

/var/folders/c8/j_nr7fhx4w9bxy7s92tnbphw0000gn/T/ipykernel_58857/3589259298.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("combined_file.csv")


In [4]:
df.head()

,index,image_link,group_id,entity_name,item_value
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height,8 centimetre
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width,42 unknown
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height,42 centimetre
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth,42 millimetre
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth,10.50 millimetre


In [5]:
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton'},
    'maximum_weight_recommendation': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre',
        'cubic foot',
        'cubic inch',
        'cup',
        'decilitre',
        'fluid ounce',
        'gallon',
        'imperial gallon',
        'litre',
        'microlitre',
        'millilitre',
        'pint',
        'quart'}
}

allowed_units = {unit for entity in entity_unit_map for unit in entity_unit_map[entity]}

In [6]:
# Function to extract and validate numeric value and unit
def process_item_value(row):
    entity = row['entity_name']
    value = str(row['item_value'])  # Ensure value is a string
    
    # Extract numeric part and unit
    match = re.match(r"([\d.]+)?\s*(.*)", value)
    if not match:
        return value  # If no match, return the original value
    
    numeric_value, unit = match.groups()
    
    # Check if numeric_value is None or empty, indicating no numeric value was provided
    if not numeric_value or not numeric_value.strip():
        numeric_value = '0.0'
    
    numeric_value = float(numeric_value)  # Convert to float
    
    # Normalize unit and check if it's valid
    unit = unit.strip().lower()
    if unit not in allowed_units:
        unit = ''  # Remove invalid units
    
    return f"{numeric_value} {unit}"

# Apply the processing function
df['item_value'] = df.apply(process_item_value, axis=1)

In [7]:
# Regular expression pattern to match: number, space, and unit
pattern = r'^\d+(\.\d+)?\s[a-zA-Z]+$'

# Filter rows that do not match the pattern
non_matching_rows = df[~df['item_value'].str.match(pattern, na=False)]

print(non_matching_rows.sample(10))

         index                                         image_link  group_id  \
121358  121448  https://m.media-amazon.com/images/I/71c4caLU+Q...    953313   
123840  123932  https://m.media-amazon.com/images/I/71qSvA1qUX...    275506   
107178  107254  https://m.media-amazon.com/images/I/61Fcu5V2rA...    558806   
106406  106482  https://m.media-amazon.com/images/I/61FA0toFI7...    347320   
88625    88697  https://m.media-amazon.com/images/I/614j5OYlO8...    243137   
115019  115099  https://m.media-amazon.com/images/I/7126ljSyMq...    939068   
110468  110547  https://m.media-amazon.com/images/I/61Wmk91qiU...    275506   
10665    10665  https://m.media-amazon.com/images/I/41VfxoAMAc...    347320   
23865    23865  https://m.media-amazon.com/images/I/513SlCOXIr...    695925   
6506      6506  https://m.media-amazon.com/images/I/41GpU3rinZ...    210293   

                          entity_name item_value  
121358                    item_weight       0.0   
123840                    it

In [8]:
# Regular expression pattern to match only numbers
pattern = r'^\d+(\.\d+)?$'

# Function to append 'inch' to numeric values
def append_unit(value):
    value_str = str(value).strip()  # Convert to string and strip whitespace
    if re.fullmatch(pattern, value_str):  # Use fullmatch to ensure the entire string matches the pattern
        return f"{value_str} inch"
    return value

# Apply the function to the item_value column
df['item_value'] = df['item_value'].apply(append_unit)

print(df)

         index                                         image_link  group_id  \
0            0  https://m.media-amazon.com/images/I/110EibNycl...    156839   
1            1  https://m.media-amazon.com/images/I/11TU2clswz...    792578   
2            2  https://m.media-amazon.com/images/I/11TU2clswz...    792578   
3            3  https://m.media-amazon.com/images/I/11TU2clswz...    792578   
4            4  https://m.media-amazon.com/images/I/11gHj8dhhr...    792578   
...        ...                                                ...       ...   
131182  131283  https://m.media-amazon.com/images/I/A1rVsIzEtk...    721522   
131183  131284  https://m.media-amazon.com/images/I/A1rdvZ5zDd...    603688   
131184  131285  https://m.media-amazon.com/images/I/A1rdvZ5zDd...    603688   
131185  131286  https://m.media-amazon.com/images/I/A1tnTUPyr7...    853009   
131186  131287  https://m.media-amazon.com/images/I/A1tnTUPyr7...    853009   

                          entity_name       item_va

In [9]:
df.head()

,index,image_link,group_id,entity_name,item_value
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height,8.0 centimetre
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width,42.0 inch
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height,42.0 centimetre
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth,42.0 millimetre
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth,10.5 millimetre


In [10]:
# Regular expression pattern to extract the unit part
unit_pattern = r'\s+([a-zA-Z]+)$'

# Extract units from the filtered data
units = df['item_value'].str.extract(unit_pattern)[0]

# Create a set of unique units
unique_units = set(units.dropna())
unique_units

{'centimetre',
 'gram',
 'inch',
 'kilogram',
 'millilitre',
 'millimetre',
 'volt',
 'watt'}

In [11]:
df.columns

Index(['index', 'image_link', 'group_id', 'entity_name', 'item_value'], dtype='object')

In [12]:
df.head()

,index,image_link,group_id,entity_name,item_value
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height,8.0 centimetre
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width,42.0 inch
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height,42.0 centimetre
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth,42.0 millimetre
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth,10.5 millimetre


In [13]:
import pandas as pd

# Create a new DataFrame with only the index and item_value columns
df_filtered = pd.DataFrame({
    'index': df['index'],
    'prediction': df['item_value']
})

# Save to CSV file
df_filtered.to_csv('submission.csv', index=True)

print("DataFrame saved to 'submission.csv'.")

DataFrame saved to 'submission.csv'.


In [14]:
df_last_two_columns = df_filtered.iloc[:, -2:].rename(columns={df_filtered.columns[-1]: 'prediction'})

# Save to CSV file
df_last_two_columns.to_csv('submission.csv', index=False)

print("DataFrame with last two columns saved to 'submission.csv'.")

DataFrame with last two columns saved to 'submission.csv'.


In [15]:
def check_file(filename):
    if not filename.lower().endswith('.csv'):
        raise ValueError("Only CSV files are allowed.")
    if not os.path.exists(filename):
        raise FileNotFoundError("Filepath: {} invalid or not found.".format(filename))

def common_mistake(unit):
    if unit in allowed_units:
        return unit
    if unit.replace('ter', 'tre') in allowed_units:
        return unit.replace('ter', 'tre')
    if unit.replace('feet', 'foot') in allowed_units:
        return unit.replace('feet', 'foot')
    return unit
    
def parse_string(s):
    s_stripped = "" if s==None or str(s)=='nan' else s.strip()
    if s_stripped == "":
        return None, None
    pattern = re.compile(r'^-?\d+(\.\d+)?\s+[a-zA-Z\s]+$')
    if not pattern.match(s_stripped):
        raise ValueError("Invalid format in {}".format(s))
    parts = s_stripped.split(maxsplit=1)
    number = float(parts[0])
    unit = common_mistake(parts[1])
    if unit not in allowed_units:
        raise ValueError("Invalid unit [{}] found in {}. Allowed units: {}".format(
            unit, s, allowed_units))
    return number, unit

def sanity_check(test_filename, output_filename):
    check_file(test_filename)
    check_file(output_filename)
    
    try:
        test_df = pd.read_csv(test_filename)
        output_df = pd.read_csv(output_filename)
    except Exception as e:
        raise ValueError(f"Error reading the CSV files: {e}")
    
    if 'index' not in test_df.columns:
        raise ValueError("Test CSV file must contain the 'index' column.")
    
    if 'index' not in output_df.columns or 'prediction' not in output_df.columns:
        raise ValueError("Output CSV file must contain 'index' and 'prediction' columns.")
    
    missing_index = set(test_df['index']).difference(set(output_df['index']))
    if len(missing_index) != 0:
        print("Missing index in test file: {}".format(missing_index))
        
    extra_index = set(output_df['index']).difference(set(test_df['index']))
    if len(extra_index) != 0:
        print("Extra index in test file: {}".format(extra_index))
    output_df.apply(lambda x: parse_string(x['prediction']), axis=1)
    print("Parsing successfull for file: {}".format(output_filename))

In [16]:
sanity_check("test.csv", "submission.csv")

Missing index in test file: {87458, 87459, 87460, 87461, 87462, 87463, 87464, 87465, 87466, 87467, 87468, 87469, 87470, 87471, 87472, 87473, 87474, 87475, 87476, 87477, 87478, 87479, 87480, 87481, 87482, 87483, 87484, 87485, 87486, 87487, 87488, 87489, 87490, 87491, 87492, 87493, 87494, 87495, 87496, 87497, 87498, 87499, 87500, 87501, 87502, 87503, 87504, 87505, 87506, 87507, 87508, 87509, 87510, 87511, 87512, 87513, 87514, 87515, 87516, 87517, 87518, 87519, 87520, 87521, 87522, 87523, 87524, 87525, 87526, 87527, 87528}
Extra index in test file: {3842, 4613, 66951, 66952, 34057, 43657, 33803, 33804, 33805, 20238, 20239, 16144, 58364, 3864, 3865, 25115, 25116, 1950, 1787, 8480, 36254, 36255, 10662, 64168, 31018, 31019, 15662, 86447, 63792, 86448, 62330, 15540, 15541, 55990, 2871, 27580, 73662, 63807, 18752, 17479, 17480, 69367, 3150, 54862, 64848, 47185, 18514, 18515, 69368, 71253, 75610, 75611, 75612, 34922, 39917, 16751, 19567, 19568, 63346, 21619, 21620, 21621, 24950, 24951, 63347, 6

In [17]:
sanity_check("test.csv", "output.csv")

Parsing successfull for file: output.csv


In [18]:
second = pd.read_csv("submission.csv")

In [19]:
first = pd.read_csv("output.csv")

In [20]:
first["index"] 

0              0
1              1
2              2
3              3
4              4
           ...  
131182    131283
131183    131284
131184    131285
131185    131286
131186    131287
Name: index, Length: 131187, dtype: int64

In [21]:
import pandas as pd

# Step 1: Read the test and reference files
# Assuming 'index' is a column in both files
test_df = pd.read_csv('original_submission.csv')  # Modify with your actual file path
reference_df = pd.read_csv('output.csv')  # Modify with your actual file path

# Step 2: Extract the indices from both files
test_indices = set(test_df['index'])
reference_indices = set(reference_df['index'])

# Given extra and missing indices in your example:
extra_indices = {3842, 4613, 66951, 66952, 34057, 43657, 33803, 33804, 33805, 20238, 
                 20239, 16144, 58364, 3864, 3865, 25115, 25116, 1950, 1787, 8480, 36254, 
                 36255, 10662, 64168, 31018, 31019, 15662, 86447, 63792, 86448, 62330, 
                 15540, 15541, 55990, 2871, 27580, 73662, 63807, 18752, 17479, 17480, 
                 69367, 3150, 54862, 64848, 47185, 18514, 18515, 69368, 71253, 75610, 
                 75611, 75612, 34922, 39917, 16751, 19567, 19568, 63346, 21619, 21620, 
                 21621, 24950, 24951, 63347, 69369, 15098, 763, 1788, 58365, 58366}

missing_indices = {87458, 87459, 87460, 87461, 87462, 87463, 87464, 87465, 87466, 87467, 
                   87468, 87469, 87470, 87471, 87472, 87473, 87474, 87475, 87476, 87477, 
                   87478, 87479, 87480, 87481, 87482, 87483, 87484, 87485, 87486, 87487, 
                   87488, 87489, 87490, 87491, 87492, 87493, 87494, 87495, 87496, 87497, 
                   87498, 87499, 87500, 87501, 87502, 87503, 87504, 87505, 87506, 87507, 
                   87508, 87509, 87510, 87511, 87512, 87513, 87514, 87515, 87516, 87517, 
                   87518, 87519, 87520, 87521, 87522, 87523, 87524, 87525, 87526, 87527, 
                   87528}

# Step 3: Replace extra indices with missing indices
# We zip the extra and missing sets to replace each extra index with a missing one.
replacement_dict = dict(zip(extra_indices, missing_indices))

# Replace the extra indices in the test DataFrame
test_df['index'] = test_df['index'].replace(replacement_dict)

# Step 4: Reorganize by ascending order of 'index'
test_df_sorted = test_df.sort_values(by='index')

# Step 5: Save the updated DataFrame back to a CSV file
test_df_sorted.to_csv('updated_test_file.csv', index=False)

print("Updated DataFrame saved to 'updated_test_file.csv'.")

Updated DataFrame saved to 'updated_test_file.csv'.


In [22]:
sanity_check("test.csv", "updated_test_file.csv")

Parsing successfull for file: updated_test_file.csv


In [23]:
sanity_check("test.csv", "updated_output_new_2.csv")

Missing index in test file: {131217, 131218, 131219, 131220, 131221, 131222, 131223, 131224, 131225, 131226, 131227, 131228, 131229, 131230, 131231, 131232, 131233, 131234, 131235, 131236, 131237, 131238, 131239, 131240, 131241, 131242, 131243, 131244, 131245, 131246, 131247, 131248, 131249, 131250, 131251, 131252, 131253, 131254, 131255, 131256, 131257, 131258, 131259, 131260, 131261, 131262, 131263, 131264, 131265, 131266, 131267, 131268, 131269, 131270, 131271, 131272, 131273, 131274, 131275, 131276, 131277, 131278, 131279, 131280, 131281, 131282, 131283, 131284, 131285, 131286, 131287}
Parsing successfull for file: updated_output_new_2.csv


In [24]:
df = pd.read_csv("updated_output_new_2.csv")

In [25]:
df.head()

,index,prediction
0,0,8.0 centimetre
1,1,42.0 inch
2,2,42.0 centimetre
3,3,42.0 millimetre
4,4,10.5 millimetre


In [36]:
# Step 1: Load the original DataFrame, ensuring no extra index column is added
df = pd.read_csv('updated_output_new_2.csv')  # Use index_col=0 to skip the first column if it's an unwanted index

# Step 2: Create a new DataFrame for the missing indices with prediction '0 inch'
missing_indices = [
    131217, 131218, 131219, 131220, 131221, 131222, 131223, 131224, 131225, 
    131226, 131227, 131228, 131229, 131230, 131231, 131232, 131233, 131234, 
    131235, 131236, 131237, 131238, 131239, 131240, 131241, 131242, 131243, 
    131244, 131245, 131246, 131247, 131248, 131249, 131250, 131251, 131252, 
    131253, 131254, 131255, 131256, 131257, 131258, 131259, 131260, 131261, 
    131262, 131263, 131264, 131265, 131266, 131267, 131268, 131269, 131270, 
    131271, 131272, 131273, 131274, 131275, 131276, 131277, 131278, 131279, 
    131280, 131281, 131282, 131283, 131284, 131285, 131286, 131287
]

missing_data = pd.DataFrame({
    'index': missing_indices,
    'prediction': ['0 inch'] * len(missing_indices)
})

# Step 3: Concatenate the original DataFrame with the missing data DataFrame
combined_df = pd.concat([df, missing_data])

# Step 4: Sort the combined DataFrame by 'index'
combined_df = combined_df.sort_values(by='index').reset_index(drop=False)  # Remove old index

# Step 5: Ensure only 'index' and 'prediction' columns are present
combined_df = combined_df[['index', 'prediction']]  # Keep only 'index' and 'prediction' columns

# Step 6: Save the resulting DataFrame if needed
combined_df.to_csv("final_submission.csv", index=False)

combined_df.head()  # Display first few rows of the sorted DataFrame

,index,prediction
0,0,8.0 centimetre
1,1,42.0 inch
2,2,42.0 centimetre
3,3,42.0 millimetre
4,4,10.5 millimetre


In [39]:
sanity_check("test.csv", "output_submission_1.csv")

Missing index in test file: {131187, 131188, 131189, 131190, 131191, 131192, 131193, 131194, 131195, 131196, 131197, 131198, 131199, 131200, 131201, 131202, 131203, 131204, 131205, 131206, 131207, 131208, 131209, 131210, 131211, 131212, 131213, 131214, 131215, 131216, 131217, 131218, 131219, 131220, 131221, 131222, 131223, 131224, 131225, 131226, 131227, 131228, 131229, 131230, 131231, 131232, 131233, 131234, 131235, 131236, 131237, 131238, 131239, 131240, 131241, 131242, 131243, 131244, 131245, 131246, 131247, 131248, 131249, 131250, 131251, 131252, 131253, 131254, 131255, 131256, 131257, 131258, 131259, 131260, 131261, 131262, 131263, 131264, 131265, 131266, 131267, 131268, 131269, 131270, 131271, 131272, 131273, 131274, 131275, 131276, 131277, 131278, 131279, 131280, 131281, 131282, 131283, 131284, 131285, 131286, 131287}
Extra index in test file: {4613, 121862, 33803, 33804, 33805, 25115, 25116, 124444, 124445, 108082, 108083, 17479, 17480, 3150, 54862, 47185, 18514, 18515, 104531,

In [41]:
# Step 1: Read the test and reference files
# Assuming 'index' is a column in both files
test_df = pd.read_csv('original_submission.csv')  # Modify with your actual file path
reference_df = pd.read_csv('output.csv')  # Modify with your actual file path

# Step 2: Extract the indices from both files
test_indices = set(test_df['index'])
reference_indices = set(reference_df['index'])
# Define the sets of missing and extra indices
missing_indices = {131187, 131188, 131189, 131190, 131191, 131192, 131193, 131194, 131195, 131196, 131197, 131198, 131199, 131200, 131201, 131202, 131203, 131204, 131205, 131206, 131207, 131208, 131209, 131210, 131211, 131212, 131213, 131214, 131215, 131216, 131217, 131218, 131219, 131220, 131221, 131222, 131223, 131224, 131225, 131226, 131227, 131228, 131229, 131230, 131231, 131232, 131233, 131234, 131235, 131236, 131237, 131238, 131239, 131240, 131241, 131242, 131243, 131244, 131245, 131246, 131247, 131248, 131249, 131250, 131251, 131252, 131253, 131254, 131255, 131256, 131257, 131258, 131259, 131260, 131261, 131262, 131263, 131264, 131265, 131266, 131267, 131268, 131269, 131270, 131271, 131272, 131273, 131274, 131275, 131276, 131277, 131278, 131279, 131280, 131281, 131282, 131283, 131284, 131285, 131286, 131287}

extra_indices = {4613, 121862, 33803, 33804, 33805, 25115, 25116, 124444, 124445, 108082, 108083, 17479, 17480, 3150, 54862, 47185, 18514, 18515, 104531, 71253, 88676, 119400, 119401, 34922, 19567, 19568, 21619, 21620, 21621, 127615, 120450, 120451, 127110, 43657, 101003, 64168, 15540, 15541, 55990, 122084, 69367, 69368, 69369, 15098, 1787, 763, 1788, 3842, 34057, 120077, 20238, 20239, 16144, 120078, 124692, 124693, 3864, 3865, 8480, 31018, 31019, 15662, 63792, 120112, 120113, 2871, 63807, 18752, 64848, 109399, 75610, 75611, 75612, 112480, 121198, 16751, 121199, 63346, 63347, 24950, 24951, 62330, 66951, 66952, 1950, 36254, 36255, 10662, 86447, 86448, 89529, 89530, 27580, 129532, 73662, 128461, 39917, 129531, 58364, 58365, 58366}

# Compare lengths of the sets
same_length = len(missing_indices) == len(extra_indices)

# Print the result
print(f"Do both sets have the same length? {same_length}")

replacement_dict = dict(zip(extra_indices, missing_indices))

# Replace the extra indices in the test DataFrame
test_df['index'] = test_df['index'].replace(replacement_dict)

# Step 4: Reorganize by ascending order of 'index'
test_df_sorted = test_df.sort_values(by='index')

# Step 5: Save the updated DataFrame back to a CSV file
test_df_sorted.to_csv('updated_test_file.csv', index=False)

print("Updated DataFrame saved to 'updated_test_file.csv'.")

Do both sets have the same length? True
Updated DataFrame saved to 'updated_test_file.csv'.


In [42]:
sanity_check("test.csv", "updated_test_file.csv")

Missing index in test file: {87458, 87459, 87460, 87461, 87462, 87463, 87464, 87465, 87466, 87467, 87468, 87469, 87470, 87471, 87472, 87473, 87474, 87475, 87476, 87477, 87478, 87479, 87480, 87481, 87482, 87483, 87484, 87485, 87486, 87487, 87488, 87489, 87490, 87491, 87492, 87493, 87494, 87495, 87496, 87497, 87498, 87499, 87500, 87501, 87502, 87503, 87504, 87505, 87506, 87507, 87508, 87509, 87510, 87511, 87512, 87513, 87514, 87515, 87516, 87517, 87518, 87519, 87520, 87521, 87522, 87523, 87524, 87525, 87526, 87527, 87528}
Parsing successfull for file: updated_test_file.csv
